In [25]:
import numpy as np
import torch
import torchvision
from torchvision.datasets import FashionMNIST

In [33]:

from torch import nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize, Compose
import os
import matplotlib.pyplot as plt


In [9]:
A = [[6, 9, 2],
     [3, 3, 7],
     [1, 0, 3]]

A_tensor = torch.tensor(A)

In [10]:
A_tensor

tensor([[6, 9, 2],
        [3, 3, 7],
        [1, 0, 3]])

In [11]:
B = np.array([0,1,2,3])
B_tensor = torch.from_numpy(B)

In [12]:
B_tensor

tensor([0, 1, 2, 3], dtype=torch.int32)

In [13]:
C = torch.zeros(4,4)
C

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [14]:
D = torch.ones(3,3,2, dtype=torch.int)
D

tensor([[[1, 1],
         [1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1],
         [1, 1]]], dtype=torch.int32)

In [15]:
A_tensor_zeros = torch.zeros_like(A_tensor)
A_tensor_zeros

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [16]:
A_tensor_rand = torch.rand_like(A_tensor, dtype=torch.float)
A_tensor_rand

tensor([[0.7967, 0.4561, 0.8299],
        [0.2551, 0.3260, 0.4388],
        [0.5600, 0.6484, 0.3430]])

In [19]:
A_tensor_rand.dtype

torch.float32

In [20]:
A_tensor_rand.shape

torch.Size([3, 3])

In [21]:
A_tensor_rand.device

device(type='cpu')

In [29]:
training_data = FashionMNIST(
    # the directory you want to store the dataset, can be a string e.g. "data"
    root = 'test_data',
    # if set to False, will give you the test set instead
    train = True,
    # download the dataset if it's not already available in the root path you specified
    download = True,
    # as the name implies, will transform images to tensor data structures so PyTorch can use them for training
    transform = torchvision.transforms.ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting test_data\FashionMNIST\raw\train-images-idx3-ubyte.gz to test_data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting test_data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to test_data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting test_data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to test_data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting test_data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to test_data\FashionMNIST\raw



In [30]:
training_data.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [31]:
training_data.class_to_idx # get the corresponding index with each class


{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [32]:
torch.cuda.is_available()

False

In [34]:
transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5),  # mean
               (0.5, 0.5, 0.5))] # std. deviation
)

In [35]:
training_data = CIFAR10(root="cifar",
                        train = True, # train set, 50k images
                        download = True,
                        transform=transform)
test_data = CIFAR10(root = "cifar",
                    train = False, # test set, 10k images
                    download = True,
                    transform = transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting cifar\cifar-10-python.tar.gz to cifar
Files already downloaded and verified


In [36]:
training_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: cifar
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [37]:
batch_size = 4
train_dataloader = DataLoader(training_data,
                              batch_size=batch_size,
                              shuffle=True)
test_dataloader = DataLoader(test_data,
                             batch_size=batch_size,
                             shuffle=True)

In [39]:
for X, y in train_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break
# Shape of X [N, C, H, W]: torch.Size([4, 3, 32, 32])
# Shape of y: torch.Size([4]) torch.int64

Shape of X [N, C, H, W]: torch.Size([4, 3, 32, 32])
Shape of y: torch.Size([4]) torch.int64


In [58]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32*32*3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)
#NeuralNetwork(
#  (flatten): Flatten(start_dim=1, end_dim=-1)
#  (linear_relu_stack): Sequential(
#    (0): Linear(in_features=3072, out_features=1024, bias=True)
#    (1): ReLU()
#    (2): Linear(in_features=1024, out_features=512, bias=True)
#    (3): ReLU()
#    (4): Linear(in_features=512, out_features=10, bias=True)
#  )
#)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [59]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD( model.parameters(), lr=0.001 ) # momentum=0.9

In [60]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 2000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [61]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X, y
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [62]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297230  [    0/50000]
loss: 1.935614  [ 8000/50000]


KeyboardInterrupt: 

In [56]:
correct = 0
total = 0
with torch.no_grad():
   for data in test_dataloader:
     images, labels = data
     outputs = model(images)
     _, predicted = torch.max(outputs.data, 1)
     total += labels.size(0)
     correct += (predicted == labels).sum().item()

print(f'Model accuracy: {100 * correct // total} %')
# Model accuracy: 53 %

Model accuracy: 53 %


In [57]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0  for classname in classes}
with torch.no_grad():
  for data in test_dataloader:
    images, labels = data
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)
    for label,prediction in zip(labels, predictions):
      if label == prediction:
        correct_pred[classes[label]] += 1
      total_pred[classes[label]] += 1
for classname, correct_count in correct_pred.items():
  accuracy = 100 * float(correct_count) / total_pred[classname]
  print(f'Accuracy for class {classname:5s}: {accuracy:.1f}%')


Accuracy for class airplane: 53.7%
Accuracy for class automobile: 62.9%
Accuracy for class bird : 35.6%
Accuracy for class cat  : 37.8%
Accuracy for class deer : 50.6%
Accuracy for class dog  : 35.9%
Accuracy for class frog : 59.9%
Accuracy for class horse: 63.6%
Accuracy for class ship : 71.8%
Accuracy for class truck: 61.0%
